## Installs and imports

In [1]:
!pip install pyscf qiskit-nature qiskit-algorithms qiskit-nature-pyscf

In [1]:
from pyscf import gto, scf, ao2mo
import numpy as np
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.hamiltonians import ElectronicEnergy
from qiskit_nature.second_q.drivers import PySCFDriver
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from qiskit_aer import Aer
from qiskit_algorithms import VQE
from qiskit.circuit.library import TwoLocal
from qiskit.quantum_info import SparsePauliOp
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit.primitives import Estimator
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_algorithms.optimizers import SPSA
from qiskit_nature.second_q.algorithms import QEOM, GroundStateSolver
from IPython.display import Image
from qiskit_algorithms import VQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.problems import BaseProblem
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.library import QFT
from qiskit.quantum_info import SparsePauliOp
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

## Construct QPE circuit for large molecules

In [3]:
def get_fermionic_op(atom_string: str, spin: int, charge: int, basis: str = "sto-3g"):
    driver = PySCFDriver(
        atom=atom_string,
        basis=basis,
        charge=charge,
        spin=spin,
        unit=DistanceUnit.ANGSTROM,
    )
    problem = driver.run()

    # Convert to FermionicOp
    atom_hamiltonian = problem.second_q_ops()

    return atom_hamiltonian[0]

In [4]:
StringLi2S = "Li 0.25 0.25 0.25; Li 0.75 0.75 0.75; S 0 0 0"

StringH2 = "H 0 0 0; H 0 0 0.735"

mol_op = get_fermionic_op(
    atom_string = StringH2,
    spin        = 0,
    charge      = 0
)

In [5]:
mapper   = JordanWignerMapper()
qubit_op = mapper.map(mol_op)

In [12]:
def generate_controlled_time_evolution(hamiltonian: SparsePauliOp, time: float):
    n_qubits = hamiltonian.num_qubits
    ctrl     = QuantumRegister(1, name='ctrl')
    system   = QuantumRegister(n_qubits, name='sys')
    circuit  = QuantumCircuit(ctrl, system)

    for pauli_string, coeff in zip(hamiltonian.paulis, hamiltonian.coeffs):
        # For each Pauli term, we build a controlled e^{-i c P t}
        pauli_label = pauli_string.to_label()
        theta = -2 * coeff.real * time  # Note: Qiskit uses exp(-i θ/2 * P) convention

        # Basis change to Z
        for i, p in enumerate(pauli_label):
            if p == 'X':
                circuit.h(system[i])
            elif p == 'Y':
                circuit.sdg(system[i])
                circuit.h(system[i])
            # If 'Z' or 'I', no basis change needed

        # Controlled multi-Z rotation
        qubit_indices = [i for i, p in enumerate(pauli_label) if p != 'I']
        if qubit_indices:
            if len(qubit_indices) == 1:
                target = qubit_indices[0]
                circuit.cx(ctrl[0], system[target])
                circuit.rz(theta, system[target])
                circuit.cx(ctrl[0], system[target])
            else:
                # Multi-qubit controlled rotation
                for q in qubit_indices[:-1]:
                    circuit.cx(ctrl[0], system[q])
                circuit.mcx(qubit_indices[:-1], system[qubit_indices[-1]])
                circuit.rz(theta, system[qubit_indices[-1]])
                circuit.mcx(qubit_indices[:-1], system[qubit_indices[-1]])
                for q in reversed(qubit_indices[:-1]):
                    circuit.cx(ctrl[0], system[q])

        # Undo basis change
        for i, p in enumerate(pauli_label):
            if p == 'X':
                circuit.h(system[i])
            elif p == 'Y':
                circuit.h(system[i])
                circuit.s(system[i])

    return circuit

def build_qpe_circuit(qubit_hamiltonian: SparsePauliOp, n_ancillas: int = 5, t0: float = 1.0):
    """Build the full QPE circuit for a given qubit Hamiltonian."""
    n_system = qubit_hamiltonian.num_qubits

    # Create quantum registers
    ancillas = QuantumRegister(n_ancillas, name='anc')
    system = QuantumRegister(n_system, name='sys')
    classical = ClassicalRegister(n_ancillas, name='c')
    qc = QuantumCircuit(ancillas, system, classical)

    # Step 1: Hadamard gates on ancillas
    qc.h(ancillas)

    # Step 2: Prepare system (for now identity, you can load HF state here)
    # Optionally prepare a Hartree-Fock state if desired

    # Step 3: Controlled evolutions
    for k in range(n_ancillas):
        t = (2 ** k) * t0
        controlled_evo = generate_controlled_time_evolution(qubit_hamiltonian, t)
        qc.append(controlled_evo.to_gate(), [ancillas[k]] + system[:])

    # Step 4: Inverse QFT
    qc.append(QFT(num_qubits=n_ancillas, inverse=True, do_swaps=True).to_gate(), ancillas)

    # Step 5: Measure ancillas
    qc.measure(ancillas, classical)

    return qc

In [13]:
qubit_op.num_qubits

4

In [14]:
circ = build_qpe_circuit(qubit_op)
circ.draw()

┌───┐┌──────────────┐                                                »
anc_0: ┤ H ├┤0             ├────────────────────────────────────────────────»
       ├───┤│              │┌──────────────┐                                »
anc_1: ┤ H ├┤              ├┤0             ├────────────────────────────────»
       ├───┤│              ││              │┌──────────────┐                »
anc_2: ┤ H ├┤              ├┤              ├┤0             ├────────────────»
       ├───┤│              ││              ││              │┌──────────────┐»
anc_3: ┤ H ├┤              ├┤              ├┤              ├┤0             ├»
       ├───┤│              ││              ││              ││              │»
anc_4: ┤ H ├┤  circuit-205 ├┤              ├┤              ├┤              ├»
       └───┘│              ││  circuit-208 ││              ││              │»
sys_0: ─────┤1             ├┤1             ├┤1 circuit-211 ├┤1             ├»
            │              ││              ││              ││  circuit-214 │»
sys_1: ─────┤2             ├┤2             ├┤2             ├┤2             ├»
            │              ││              ││              ││              │»
sys_2: ─────┤3             ├┤3             ├┤3             ├┤3             ├»
            │              ││              ││              ││              │»
sys_3: ─────┤4             ├┤4             ├┤4             ├┤4             ├»
            └──────────────┘└──────────────┘└──────────────┘└──────────────┘»
  c: 5/═════════════════════════════════════════════════════════════════════»
                                                                            »
«                       ┌───────┐┌─┐            
«anc_0: ────────────────┤0      ├┤M├────────────
«                       │       │└╥┘┌─┐         
«anc_1: ────────────────┤1      ├─╫─┤M├─────────
«                       │       │ ║ └╥┘┌─┐      
«anc_2: ────────────────┤2 IQFT ├─╫──╫─┤M├──────
«                       │       │ ║  ║ └╥┘┌─┐   
«anc_3: ────────────────┤3      ├─╫──╫──╫─┤M├───
«       ┌──────────────┐│       │ ║  ║  ║ └╥┘┌─┐
«anc_4: ┤0             ├┤4      ├─╫──╫──╫──╫─┤M├
«       │              │└───────┘ ║  ║  ║  ║ └╥┘
«sys_0: ┤1             ├──────────╫──╫──╫──╫──╫─
«       │              │          ║  ║  ║  ║  ║ 
«sys_1: ┤2 circuit-217 ├──────────╫──╫──╫──╫──╫─
«       │              │          ║  ║  ║  ║  ║ 
«sys_2: ┤3             ├──────────╫──╫──╫──╫──╫─
«       │              │          ║  ║  ║  ║  ║ 
«sys_3: ┤4             ├──────────╫──╫──╫──╫──╫─
«       └──────────────┘          ║  ║  ║  ║  ║ 
«  c: 5/══════════════════════════╩══╩══╩══╩══╩═
«                                 0  1  2  3  4

In [15]:
!pip install qsharp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 28.4 MB/s eta 0:00:00


In [16]:
from qiskit import QuantumCircuit
from qsharp.interop.qiskit import estimate

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0, 1)
qc.cx(0, 2)
qc.measure_all()
print(qc.draw())

def estimate_resources(quantum_circuit):
    result = estimate(quantum_circuit)
    return result['physicalCounts']['physicalQubits'], result['physicalCounts']['runtime']

ft_qc_qubits, ft_qc_runtime = estimate_resources(qc)
print(f"Original Circuit: Qubits = {qc.num_qubits}")
print(f"Fault Tolerant Circuit: Qubits = {ft_qc_qubits}, Runtime = {ft_qc_runtime}")

        ┌───┐           ░ ┌─┐      
   q_0: ┤ H ├──■────■───░─┤M├──────
        └───┘┌─┴─┐  │   ░ └╥┘┌─┐   
   q_1: ─────┤ X ├──┼───░──╫─┤M├───
             └───┘┌─┴─┐ ░  ║ └╥┘┌─┐
   q_2: ──────────┤ X ├─░──╫──╫─┤M├
                  └───┘ ░  ║  ║ └╥┘
meas: 3/═══════════════════╩══╩══╩═
                           0  1  2 
Original Circuit: Qubits = 3
Fault Tolerant Circuit: Qubits = 202824, Runtime = 180000


In [ ]:
ft_qc_qubits, ft_qc_runtime = estimate_resources(circ.decompose())
print(f"Original Circuit: Qubits = {circ.num_qubits}")
print(f"Fault Tolerant Circuit: Qubits = {ft_qc_qubits}, Runtime = {ft_qc_runtime}")